制作多模态预训练数据集

In [ ]:
import os
import glob
import base64
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import time
import csv

client = OpenAI(
    api_key="sk-b31b21a0de2240118273137745f5d396",  # 替换为你的 API Key
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 数据集路径
METADATA_FILE = "HAM10000_metadata.csv"
IMAGE_DIR_1 = os.path.join("data", "HAM10000", "HAM10000_images_part_1")
IMAGE_DIR_2 = os.path.join("data", "HAM10000", "HAM10000_images_part_2")

# 输出文件
DESCRIPTIONS_FILE = "ham10000_reasoning_descriptions.csv"

# 类别缩写到全称的映射，用于生成更自然的Prompt
DIAGNOSIS_MAPPING = {
    'akiec': '光化性角化病 (Actinic Keratosis)',
    'bcc': '基底细胞癌 (Basal Cell Carcinoma)',
    'bkl': '脂溢性角化病 (Benign Keratosis)',
    'df': '皮肤纤维瘤 (Dermatofibroma)',
    'mel': '黑色素瘤 (Melanoma)',
    'nv': '黑色素细胞痣 (Melanocytic Nevus)',
    'vasc': '血管性病变 (Vascular Lesion)'
}

# --- 2. 动态生成专家级Prompt的函数 ---

def create_reasoning_prompt(diagnosis_code):
    """
    根据给定的诊断代码，动态创建一个引导模型进行推理的Prompt。
    """
    diagnosis_fullname = DIAGNOSIS_MAPPING.get(diagnosis_code, "未知病变")

    prompt = f"""
假如你是一名国际顶级的皮肤病理学AI诊断专家，你将根据一张已确诊为 **'{diagnosis_fullname}'** 的皮肤镜图像，来详细解释为什么该图像符合此诊断。根据以下规则一步步执行：
1. 从总体结构与形态方面描述，阐述病变的整体形态（对称性、结构组织、大小比例），说明病灶呈现规则或不规则形态，以及整体结构如何与该诊断相符。
2. 从边缘特征方面描述，说明病变的边界是清晰、平滑、模糊还是呈现放射状延伸，阐述边界特征如何提示其为良性或恶性病变。
3. 从颜色与分布模式方面描述，指出病变中出现的颜色种类（棕色、黑色、红色、蓝灰色、白色等），分析颜色的分布是否均匀、是否存在局部集中的色素区或色调突变，阐述颜色分布如何支持该诊断。
4. 从关键皮肤镜结构与诊断依据方面描述，根据具体诊断类别，识别和描述最具代表性的皮肤镜结构与模式：
    - 若诊断为黑色素瘤 (mel, Melanoma)，需描述明显的不对称性、不规则边缘、颜色多样性（棕、黑、蓝、白、红等），非典型色素网络、蓝白幕、放射状线条、负网状结构、不对称的小点或条纹、局部回避区等恶性特征。
    - 若诊断为基底细胞癌 (bcc, Basal Cell Carcinoma)，需描述树枝状血管、蓝灰色卵圆巢、光滑珠光边缘、溃疡或结痂区域、车轮辐射状结构、白色条纹或亮点。
    - 若诊断为黑色素细胞痣 (nv, Melanocytic Nevus)，需描述整体对称、规则的色素网络、均匀的棕色色调、清晰边界，可见规则点状或球状结构、均匀分布的色素网格。
    - 若诊断为脂溢性角化病 (bkl, Benign Keratosis)，需描述粉刺样开口、脑回状（丘脑状）结构、粘贴感外观、白色假网状结构、角质栓、黑点或伪毛囊口。
    - 若诊断为光化性角化病 (akiec, Actinic Keratosis)，需描述红白交错的表面、毛细血管扩张、鳞屑、角质过度增生、淡棕或红色调，可能可见“草地样”或“红白斑块状”结构。
    - 若诊断为皮肤纤维瘤 (df, Dermatofibroma)，需描述中心棕色区伴周围淡色晕、放射状色素结构、中心瘢痕样白区、周边色素网络逐渐消退、轻微凹陷。
    - 若诊断为血管性病变 (vasc, Vascular Lesion)，需描述均匀的红色至紫色区域、清晰可见的血管结构、点状或线状血管、湖状血管样分布、整体对称。

参考例子：无

请回答问题：
诊断名称：**'{diagnosis_fullname}'**
输出：

要求：
1 用简洁、条理清晰的专家语言作答。
2 回答应聚焦于图像可见特征与诊断逻辑的对应关系，避免空泛描述或重复结论。
"""
    return prompt

# --- 3. 图像处理函数 ---

def get_image_reasoning(image_path, prompt):
    """调用VLM模型为单个图像生成推理式描述"""
    try:
        with open(image_path, 'rb') as img_file:
            img_base64 = base64.b64encode(img_file.read()).decode('utf-8')
        data_url = f"data:image/jpeg;base64,{img_base64}"
        
        completion = client.chat.completions.create(
            model="qwen2.5-vl-32b-instruct",
            messages=[{
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": data_url}}
                ]
            }]
        )
        description = completion.choices[0].message.content.strip()
        # 清理文本，便于CSV存储
        return description.replace("\n", " ").replace('"', "'").replace(",", ";")
    except Exception as e:
        print(f"\n处理图像 {os.path.basename(image_path)} 时发生API错误: {e}")
        return "ERROR_API_CALL"



In [ ]:
# --- 4. 主执行逻辑 ---
print("--- 第一步 (优化版): 生成基于诊断的推理式描述 ---")
metadata_df = pd.read_csv(METADATA_FILE)

print("正在构建图像路径索引...")
all_image_paths = glob.glob(os.path.join(IMAGE_DIR_1, '*.jpg')) + \
                  glob.glob(os.path.join(IMAGE_DIR_2, '*.jpg'))
image_path_map = {os.path.splitext(os.path.basename(p))[0]: p for p in all_image_paths}

# 断点续传逻辑
completed_image_ids = set()
if os.path.exists(DESCRIPTIONS_FILE):
    try:
        desc_df = pd.read_csv(DESCRIPTIONS_FILE)
        if 'image_id' in desc_df.columns:
            completed_image_ids = set(desc_df['image_id'])
        print(f"检测到已有描述文件，已处理 {len(completed_image_ids)} 张图像，将从断点继续。")
    except Exception:
        print(f"描述文件 '{DESCRIPTIONS_FILE}' 读取异常，将从头开始。")
# 准备写入文件
with open(DESCRIPTIONS_FILE, 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    if not completed_image_ids:
        writer.writerow(["image_id", "dx", "reasoning_description"])
    # 遍历元数据
    for _, row in tqdm(metadata_df.iterrows(), total=metadata_df.shape[0], desc="生成推理描述"):
        image_id = row['image_id']
        dx = row['dx']
        
        if image_id in completed_image_ids:
            continue
        image_path = image_path_map.get(image_id)
        if not image_path:
            continue
        
        # 1. 动态生成针对该图片的Prompt
        prompt = create_reasoning_prompt(dx)
        
        # 2. 调用API获取推理描述
        reasoning = get_image_reasoning(image_path, prompt)
        
        # 3. 立即写入结果
        writer.writerow([image_id, dx, reasoning])
        
        # 4. API调用频率控制
        time.sleep(1) 
print(f"\n--- 所有推理式描述已成功生成并保存至 '{DESCRIPTIONS_FILE}' ---")